# Reproducibility study - Counterfactual Generative Networks


## Setup
We first need to install the packages 

In [2]:
import sys
py = sys.executable
!{sys.executable} -m pip install -r requirements.txt

     |████████████████████████████████| 881.9 MB 7.9 kB/s 
     |████████████████████████████████| 23.3 MB 16.2 MB/s 
     |████████████████████████████████| 15.4 MB 12.4 MB/s 
ERROR: rasterio 1.2.3 has requirement click<8,>=4.0, but you'll have click 8.0.1 which is incompatible.
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cpu
    Uninstalling torch-1.9.0+cpu:
      Successfully uninstalled torch-1.9.0+cpu
  Attempting uninstall: numpy
    Found existing installation: numpy 1.17.2
    Uninstalling numpy-1.17.2:
      Successfully uninstalled numpy-1.17.2
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.3
    Uninstalling torchvision-0.11.3:
      Successfully uninstalled torchvision-0.11.3


In [5]:
import gdown

gdown.download('https://drive.google.com/u/0/uc?export=download&confirm=rHtT&id=1NSv4RCSHjcHois3dXjYw_PaLIoVlLgXu', 'colored_mnist.tar.gz')
gdown.download("https://drive.google.com/u/0/uc?id=1VkKexkWh5SeB8fgxAZxLKgmmvDXhVYUy&export=downloadl", "u2net.pth")
gdown.download("https://drive.google.com/u/0/uc?id=12yVFHPUjKmUFGnO2D4xVlTSpF8CUj136&export=download", "cgn.pth")


Downloading...
From: https://drive.google.com/u/0/uc?export=download&confirm=rHtT&id=1NSv4RCSHjcHois3dXjYw_PaLIoVlLgXu
To: /mnt/c/Users/Mundv/test3/FACT_CGN/colored_mnist.tar.gz
100%|██████████| 38.8M/38.8M [00:02<00:00, 15.0MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=1VkKexkWh5SeB8fgxAZxLKgmmvDXhVYUy&export=downloadl
To: /mnt/c/Users/Mundv/test3/FACT_CGN/u2net.pth
100%|██████████| 176M/176M [00:16<00:00, 10.7MB/s] 
Downloading...
From: https://drive.google.com/u/0/uc?id=12yVFHPUjKmUFGnO2D4xVlTSpF8CUj136&export=download
To: /mnt/c/Users/Mundv/test3/FACT_CGN/cgn.pth
 69%|██████▉   | 772M/1.11G [16:48<24:33:22, 3.88kB/s]

In [3]:
%%sh
#!/usr/bin/env bash
# Move Colored MNIST
tar -xzvf colored_mnist.tar.gz
mv colored_mnist mnists/data
rm colored_mnist.tar.gz

# Download BG challenge dataset
wget https://github.com/MadryLab/backgrounds_challenge/releases/download/data/backgrounds_challenge_data.tar.gz
tar -xzvf backgrounds_challenge_data.tar.gz
mkdir imagenet/data/in9
mv bg_challenge/* imagenet/data/in9
rmdir bg_challenge
rm backgrounds_challenge_data.tar.gz

# Download the Cue Conflict dataset
git clone https://github.com/rgeirhos/texture-vs-shape/
mkdir imagenet/data/cue_conflict
mv texture-vs-shape/stimuli/style-transfer-preprocessed-512/* imagenet/data/cue_conflict
rm -rf texture-vs-shape


Process is interrupted.


In [ ]:
#!/usr/bin/env bash

mv u2net.pth imagenet/weights

mv cgn.pth imagenet/weights

wget "https://s3.amazonaws.com/models.huggingface.co/biggan/biggan-deep-256-pytorch_model.bin"
mv biggan-deep-256-pytorch_model.bin imagenet/weights/biggan256.pth

## Experiments

### MNIST

We first need to train the CGN such that it can generate counterfactual images.

In [ ]:
!py -u mnists/train_cgn.py --cfg mnists/experiments/cgn_colored_MNIST/cfg.yaml
!py -u mnists/train_cgn.py --cfg mnists/experiments/cgn_double_colored_MNIST/cfg.yaml
!py -u mnists/train_cgn.py --cfg mnists/experiments/cgn_wildlife_MNIST/cfg.yaml

Let's generate the datasets!

In [ ]:
!py -u mnists/generate_data.py --dataset colored_MNIST
!py -u mnists/generate_data.py --weight_path mnists/experiments/cgn_colored_MNIST/weights/ckp.pth --dataset colored_MNIST --no_cfs 10 --dataset_size 100000 --counterfactual
!py -u mnists/generate_data.py --dataset wildlife_MNIST
!py mnists/generate_data.py --weight_path mnists/experiments/cgn_wildlife_MNIST/weights/ckp.pth --dataset wildlife_MNIST --no_cfs 5 --dataset_size 100000 --counterfactual
!py -u mnists/generate_data.py --dataset double_colored_MNIST
!py -u mnists/generate_data.py --weight_path mnists/experiments/cgn_double_colored_MNIST/weights/ckp.pth --dataset double_colored_MNIST --no_cfs 10 --dataset_size 100000 --counterfactual